In [1]:
from telethon import TelegramClient
import asyncio

api_id = 19890265
api_hash = '1a1fb40bc6fb9e6475f443742f4c4fe5'
phone = '+381631899159'
password = 'sact0cugh0lick!GRAW'  # Ваш пароль для 2FA

client = TelegramClient('session_name', api_id, api_hash)

async def main():
    await client.start(phone)
    if await client.is_user_authorized():
        print("Уже авторизован")
    else:
        await client.send_code_request(phone)
        code = input('Введите код: ')
        try:
            await client.sign_in(phone, code)
        except Exception as e:
            if 'PASSWORD_HASH_INVALID' in str(e):
                await client.sign_in(password=password)
    print("Авторизация успешна")

asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [5]:
import requests
from bs4 import BeautifulSoup

response = requests.get(url)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')
tables = soup.find_all('table')

In [15]:
extracted_tables = []

for tab in tables:
    # Пытаемся найти caption
    caption_tag = tab.find('caption')
    caption = caption_tag.get_text(strip=True) if caption_tag else None
    
    # Пытаемся определить заголовки столбцов:
    # Ищем thead или берем первую строку tr
    headers = []
    thead = tab.find('thead')
    if thead:
        header_row = thead.find('tr')
        if header_row:
            headers = [th.get_text(strip=True) for th in header_row.find_all(['th','td'])]
    else:
        # Если нет thead, берем первую tr в tbody или напрямую в table
        first_row = tab.find('tr')
        if first_row:
            headers = [th.get_text(strip=True) for th in first_row.find_all(['th','td'])]
    
    # Извлекаем все остальные строки
    rows = []
    all_tr = tab.find_all('tr')
    # Пропускаем первую строку если она заголовочная
    data_rows = all_tr[1:] if len(all_tr) > 1 else []
    
    for r in data_rows:
        cells = [td.get_text(strip=True) for td in r.find_all(['td','th'])]
        rows.append(cells)
    
    extracted_tables.append({
        "caption": caption,
        "headers": headers,
        "rows": rows,
    })

In [16]:
extracted_tables

[{'caption': None,
  'headers': ['Реквизиты корпоративного действия'],
  'rows': [['Референс корпоративного действия', '806764'],
   ['Код типа корпоративного действия', 'MEET'],
   ['Тип корпоративного действия', 'Годовое общее собрание акционеров'],
   ['Дата КД (факт.)', '06 июня 2023 г.'],
   ['Дата фиксации', '12 мая 2023 г.'],
   ['Форма проведения собрания', 'Заочная']]},
 {'caption': None,
  'headers': ['Информация о ценных бумагах'],
  'rows': [['Референс КД по ценной бумаге',
    'Эмитент',
    'Регистрационный номер',
    'Дата регистрации',
    'Категория',
    'Депозитарный код выпуска',
    'ISIN',
    'Реестродержатель'],
   ['806764X23120',
    'Акционерное общество "Ульяновская сетевая компания"',
    '1-01-03826-E',
    '02 октября 2006 г.',
    'акции обыкновенные',
    'RU000A0JUY39',
    'RU000A0JUY39',
    'АО "НРК - Р.О.С.Т."']]}]

In [12]:
extracted_tables

# Сохраняем результат в JSON для удобства дальнейшей ручной разметки
import json
with open("parsed_tables.json", "w", encoding="utf-8") as f:
    json.dump(extracted_tables, f, ensure_ascii=False, indent=2)
